In [71]:
from importlib.util import set_loader
from tkinter import Grid
import numpy as np
#import Pygame
import operator
import random as rd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
class world:
    def __init__(self):
        self.height = 20
        self.width = 20
        self.randomize_counter = 0 #  فقط برای اینکه هر دست بیشتر از یکبار رندوم نزنه جاهارو
        self.grid = np.zeros(( self.height, self.width))
        self.foods = np.zeros((2,8)) #لوکیشن همه غذاها
        self.agents = np.zeros((2,4)) #لوکیشن همه‌ی ایجنت ها
        
        #جای تصادفیاولیه غذا
    def make_food(self):   
    #random loc for foods
        for i in range (0,8):
            x=np.random.randint(0,19)
            rd.seed(datetime.now())
            y=np.random.randint(0,19)
            #چک کردن روی هم نیوفتادن غذا ها
            for j in range(0,i-1):
                if self.foods[0][j]== self.foods[0][i] and self.foods[1][j]== self.foods[1][i] :
                    x=np.random.randint(0,19)
                    rd.seed(datetime.now())
                    y=np.random.randint(0,19)
                    
            self.grid[x][y] = 1
            self.foods[0][i]= x
            self.foods[1][i]= y
            
    
    # جای اولیه ایجنت ها
    def agent_rand_loc(self):
        if self.randomize_counter > 1: # بررسی اینکه یه بار بیشتر باز نشه
            print("Can't randomize agent again namusan...")
        else:    
            self.randomize_counter = self.randomize_counter + 1
            for i in range (0,4):
                x=np.random.randint(0,19)
                rd.seed(datetime.now())
                y=np.random.randint(0,19)
                for j in range(0,i-1):
                    if self.agents[0][j]== self.agents[0][i] and self.agents[1][j]== self.agents[1][i] :
                        x=np.random.randint(0,19)
                        rd.seed(datetime.now())
                        y=np.random.randint(0,19)
                if self.grid[x][y]!=1 :
                    self.grid[x][y] = 2
                    self.agents[0][i]= x
                    self.agents[1][i]= y
                else:
                    rd.seed(datetime.now())
                    self.grid[x][np.random.randint(0,19)] = 2
                    self.agents[0][i]= x
                    self.agents[1][i]= np.random.randint(0,19)
    
        #وارد کردن اخرین تغییرات در ماتریس نقشه و برگردوندنش
    def matrix_world(self):
        virtual_space = np.zeros((20,20))
        if self.randomize_counter <=  1: # بررسی اینکه یه بار بیشتر رندوم نزنه 
            self.agent_rand_loc()
            self.make_food()
            self.randomize_counter = self.randomize_counter + 1
            for i in range (0,8):
                x= int(self.foods[0][i])
                y= int(self.foods[1][i])
                virtual_space[x][y] = 1
            for i in range (0,4):
                x= int(self.agents[0][i])
                y= int(self.agents[1][i])
                virtual_space[x][y] = 2
        else:
            for i in range (0,8):
                x= int(self.foods[0][i])
                y= int(self.foods[1][i])
                virtual_space[x][y] = 1
            for i in range (0,4):
                x= int(self.agents[0][i])
                y= int(self.agents[1][i])
                virtual_space[x][y] = 2
        self.grid = virtual_space
        return virtual_space
    
    
   # def get_available_act(????)

    #def end_game


In [73]:
class agent:
    
    all_agent = []
    
    def __init__(self,score,xloc,yloc):
        self.score = score
        self.xloc= xloc # محور عمودی
        self.yloc=yloc  # محور افقی
        # برای مشخص کردن برنده در زمان حمله، اگر کسی که بهش حمله میشه جهت آخرین حرکتش به سمت حمله کننده نباشه، حمله پیروز میشه.
        # پس طبق آخرین حرکت به هر ایجنت یه جهت میدیم direction
        self.direction= 1 # up:1 , down:2 , right:3 , left:4
        agent.all_agent.append(self)

    def __str__(self):
           return "{self.name}"

    
    #def update_agentloc(self):
    #    for c,j in zip(agent.all_agent,range(0,4)):
    #            gameset.agents[0][j] = c.xloc
    #            gameset.agents[1][j] = c.yloc
                
#حرکت کردن ایجنت + مشخص کردن جهت ایجنت + امتیاز دادن بهش                      
# 1(up),2(down),3(right),4(left)
    def apply_action(self,act,online_space):
        m=int(self.xloc)
        n=int(self.yloc)

        if act == 1 : #up
            if online_space[m-1][n] == 999: #رفتن به خارج از نقشه
                print("wrong direction... Try again please") #یه چیزی بزار که دوباره وارد کنه جهتو
            elif online_space[m-1][n] == 2 :    #جایزه حمله
                self.direction = 1
                self.score= self.score + 2
                self.being_attacked(m-1,n,1)
                self.xloc= self.xloc - 1
            elif online_space[m-1][n] == 1 :    #جایزه غذا
                self.score=self.score + 1
                self.xloc= self.xloc - 1
            elif online_space[m-1][n] == 0:
                self.direction = 1
                self.xloc= self.xloc - 1

  
        elif act==2 : #down
            if online_space[m+1][n] == 999:#رفتن به خارج از نقشه
                print("wrong direction... Try again please") #یه چیزی بزار که دوباره وارد کنه جهتو
            elif online_space[m+1][n] == 2 :    #جایزه حمله
                self.direction = 2
                self.score= self.score + 2
                self.being_attacked(m+1,n,2)
                self.xloc= self.xloc + 1
            elif online_space[m+1][n] == 1 :    #جایزه غذا
                self.direction = 2
                self.score = self.score + 1
                self.xloc= self.xloc + 1
            elif online_space[m+1][n] == 0:
                self.direction = 2
                self.xloc= self.xloc + 1
            
        elif act==3 : #right
            if online_space[m][n+1] == 999:#رفتن به خارج از نقشه
                print("wrong direction... Try again please") #یه چیزی بزار که دوباره وارد کنه جهتو
            elif online_space[m][n+1] == 2:
                self.direction = 3
                self.score = self.score + 2
                self.being_attacked(m,n +1,3)
                self.yloc =self.yloc + 1
            elif online_space[m][n+1] == 1 :  
                self.direction = 3
                self.score = self.score + 1
                self.yloc =self.yloc + 1
            elif online_space[m][n+1] == 0:
                self.direction = 3
                self.yloc =self.yloc + 1

        elif act==4 : #left
            if online_space[m-1][n] == 999:#رفتن به خارج از نقشه
                print("wrong direction... Try again please") #یه چیزی بزار که دوباره وارد کنه جهتو
            elif online_space[m][n-1] == 2 :    #جایزه حمله
                self.direction = 4
                self.score = self.score + 2
                self.being_attacked(m ,n-1,4)
                self.yloc= self.yloc -1
            elif online_space[m][n-1] == 1 :    #جایزه غذا
                self.direction = 4
                self.score = self.score + 1
                self.yloc= self.yloc -1
            elif online_space[m][n-1] == 0 :
                self.direction = 4
                self.yloc= self.yloc -1
                
            
        #self.update_agentloc()
        
            
# پیدا کردن ایجنتی که بهش حمله شده + بررسی جهت خودش + امتیاز دادن        
    def being_attacked(self,xloc,yloc,attacker_direc):
        for c in agent.all_agent:
            if xloc == c.xloc and yloc == c.yloc:
                if c != self :
                    if c.direction!=attacker_direc :
                        c.score = c.score - 4
                    elif c.direction == attacker_direc:
                        # remove 2 points reward and reduce 4 point more because i like...
                        self.score = self.score - 6 
                        c.score = c.score + 2
                return "defeat"
        
                       
        
        # search between users of class for find location of agent who moved 
        
            
    #def reward_checker(self,online_space):
        
    #def check_situation(self,online_space,act)
     
        
    
    
            # یه ماتریس ۳*۳ که خود ایجنت وسطشه
            # تابع برای ساخت ماتریس فضای قابل مشاهده هر ایجنت
    def vis (self,online_space):
        visi = np.zeros((3,3))
        a = int(self.xloc) -1
        b = int(self.yloc) -1
        #(i,j) شمارنده برای ساخت ماتریس ۳ در ۳ از ماتریس فضای اصلی
        #(visx,visy) شمارنده در ماتریس ویز (فضای قابل مشاهده) برای هر ایجنت
        #999 قسمت های خارج از نقشه و غیر قابل ورود
        for i,visx in zip(range(a ,a+3),range(0,3)) :
            for j,visy in zip(range(b,b+3),range(0,3)):
                    if j < 0:
                        visi[visx][visy] = 999
                    elif i< 0:
                        visi[visx][visy] = 999
                    else:
                        visi[visx][visy] = online_space[i][j]
                        
        return visi
    

In [74]:
# play game :::::::

#when click start ...

gameset = world()           
gameset.agent_rand_loc()  # فقط یک بار ران میشه که تصادفی به ایجنت جا بده                     
                            #معرفی ایجنت ها به کلاس agent
                            #وارد کردن لوکیشن و امتیاز اولیه هر کدام
for j in range(0,4):
    if j==0:
        agent1 = agent(0,gameset.agents[0][j],gameset.agents[1][j])
    elif j==1:
        agent2 = agent(0,gameset.agents[0][j],gameset.agents[1][j])
    elif j==2:
        agent3 = agent(0,gameset.agents[0][j],gameset.agents[1][j])
    elif j==3:
        agent4 = agent(0,gameset.agents[0][j],gameset.agents[1][j])
            
    
#    if ("make condition for ending of game")    باید پایان بزاری
#        print("game over looser")
#        break

In [87]:
last_space_update = gameset.matrix_world()
agent3.vis(last_space_update) # قسمتی از نقشه که این ایجنت میبینه


array([[999.,   0.,   1.],
       [999.,   0.,   0.],
       [999.,   0.,   0.]])

In [88]:

aact = 1 #جهت حرکت ایجنت
# 1(up),2(down),3(right),4(left)
# با هر بار اجرای پایینی ایجنتی که اولش نوشته شده حرکت می‌کنه و امتیاز میگیره
agent3.apply_action(aact,last_space_update)


In [89]:
# ماتریسی که فقط لوکیشن ایجنتا توشه



array([[17.,  1., 13.,  2.],
       [ 7.,  1.,  0.,  8.]])

In [69]:
agent3.score

0

In [70]:
gameset.matrix_world()


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.